In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime
plt.style.use('default')
import warnings
warnings.filterwarnings("ignore")

# DataSet
#### Read csv file
#### Create dataframe

In [6]:
dataset=pd.read_csv(r"C:\Datasets-base\Datasets\Power_Production_Solar_Panels\PV_Elec_Gas3.csv")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3304 entries, 0 to 3303
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date                    3304 non-null   object 
 1   Cumulative_solar_power  3304 non-null   float64
 2   kWh electricity/day     3304 non-null   float64
 3   Gas/day                 3304 non-null   float64
dtypes: float64(3), object(1)
memory usage: 103.4+ KB
